In [10]:
import openeo
conn = openeo.connect("https://openeo.vito.be/openeo/").authenticate_oidc()

Authenticated using refresh token.


In [27]:
properties = {"eo:cloud_cover": lambda x: x.lte(65)}
collection = "SENTINEL2_L2A"
s2_bands = ["B02","B03","B04","B05","B06","B07","B08","B8A","B11","B12","SCL"]
spatial_extent = {"west": 12.17,
                  "east": 12.37,
                  "south": 51.46,
                  "north": 51.69}
temporal_extent = ["2022-01-01","2022-12-31"]
s2_cube = conn.load_collection(
                collection,
                spatial_extent = spatial_extent,
                temporal_extent = temporal_extent,
                bands = s2_bands,
                properties = properties
)

## We use the SCL band to create a mask where clouded pixels are set to 1
## and other pixels are set to 0
scl_band = s2_cube.band("SCL")
s2_cloudmask = ( (scl_band == 8) | (scl_band == 9) | (scl_band == 3) ) * 1.0
s2_bands.remove("SCL")
s2_cube_masked = s2_cube.filter_bands(s2_bands).mask(s2_cloudmask)

# Create weekly composite
composite = s2_cube_masked.aggregate_temporal_period(
    period = "week",
    reducer = "mean"
)
# Fill gaps with linear interpolation
interpolated = composite.apply_dimension(
    dimension = "t",
    process = "array_interpolate_linear"
)

cube_feb_nov = interpolated.filter_temporal(["2022-02-01","2022-11-30"]) # This should result in 43 weeks

cube_nc = cube_feb_nov.save_result(format="netCDF")

Visit https://aai.egi.eu/device?user_code=UEOF-UVWL 📋 to authenticate.

[################################-----] ✅ Authorized successfully

Authenticated using device code flow.


In [28]:
job = cube_nc.create_job(title="test_set_pv_farms")
job.start_job()

<BatchJob job_id='j-240322045ecb41468e5b0741bae0294e'>

In [13]:
job = conn.job("j-24032286b258432b956259cd735f1ea0")

In [16]:
results = job.get_results()
results.download_files("./test_data")

[PosixPath('test_data/openEO.nc'), PosixPath('test_data/job-results.json')]

In [4]:
import xarray as xr
ds = xr.open_dataset("./test_data/openEO.nc",decode_coords="all",chunks={})
ds

<xarray.Dataset> Size: 7GB
Dimensions:  (t: 43, x: 1482, y: 2610)
Coordinates:
  * t        (t) datetime64[ns] 344B 2022-02-06 2022-02-13 ... 2022-11-27
  * x        (x) float64 12kB 3.034e+05 3.034e+05 ... 3.182e+05 3.182e+05
  * y        (y) float64 21kB 5.73e+06 5.73e+06 5.73e+06 ... 5.704e+06 5.704e+06
    crs      |S1 1B ...
Data variables:
    B02      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B03      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B04      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B05      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B06      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B07      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B08      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B8A      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B11      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    B12      (t, y, x) float32 665MB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.28.2a1
    description:  
    title:

In [5]:
# Reshape data
# Reshape to feature vectors: 10 bands * 43 weeks: 430 values
# Flatten the spatial dimensions
# From {"variable, "t", "y", "x"} to {"y"*"x", "variable", "t"} to {"y"*"x", "variable"*"t"}
ds = ds.to_dataarray()
ds_np = ds.transpose("y","x","variable","t").values.reshape(len(ds["y"])*len(ds["x"]),len(ds["variable"])*len(ds["t"]))

In [6]:
import numpy as np
ds_np = np.nan_to_num(ds_np, nan=-999999)

In [7]:
# load the model from GitHub
import pickle
import urllib.request
clf = pickle.load(urllib.request.urlopen("https://github.com/masawdah/openEO_photovoltaic/raw/main/models/temporal_models/rf_weekly_1.sav"))

In [8]:
# Predict values
te_pred = clf.predict(ds_np)

In [9]:
# Reshape resulting vector to original shape
te_pred_mat = te_pred.reshape(len(ds["y"]),len(ds["x"]))

In [30]:
# Assign result to xarray DataArray object
out_ds = ds[0,0]
out_ds.data = te_pred_mat
out_ds.rio.write_crs(ds.crs.attrs["crs_wkt"],inplace=True)

<xarray.DataArray (y: 2610, x: 1482)> Size: 31MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
    t            datetime64[ns] 8B 2022-02-06
  * x            (x) float64 12kB 3.034e+05 3.034e+05 ... 3.182e+05 3.182e+05
  * y            (y) float64 21kB 5.73e+06 5.73e+06 ... 5.704e+06 5.704e+06
    crs          |S1 1B ...
    variable     <U3 12B 'B02'
    spatial_ref  int64 8B 0
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.28.2a1
    description:  
    title:

In [32]:
# Write sample result for visual inspection
out_ds = out_ds.to_dataset(name="solar")
out_ds.rio.to_raster("sample_result.tiff",driver="COG")